## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
%tensorflow_version 2.x
import tensorflow as tf

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


In [79]:
print(tf.__version__)

2.0.0


### Collect Data

In [80]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv("/content/drive/My Drive/prices.csv", delimiter=",")

In [83]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Check all columns in the dataset

In [84]:
data.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop(columns=['date','symbol'],inplace=True)

In [86]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
df=data[:1000]

In [88]:
df.shape

(1000, 5)

In [89]:
X=df.drop(columns=['volume'])
X.head()

,open,close,low,high
0,123.430000,125.839996,122.309998,126.250000
1,125.239998,119.980003,119.940002,125.540001
2,116.379997,114.949997,114.930000,119.739998
3,115.480003,116.620003,113.500000,117.440002
4,117.010002,114.970001,114.089996,117.330002


In [90]:
y=df['volume']/1000000
y.head()

0    2.1636
1    2.3864
2    2.4895
3    2.0063
4    1.4086
Name: volume, dtype: float64

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(X,y,test_size=.30,random_state=2)

#### Convert Training and Test Data to numpy float32 arrays


In [92]:
train_x.dtypes

open     float64
close    float64
low      float64
high     float64
dtype: object

In [93]:
y.dtypes

dtype('float64')

In [0]:
import numpy as np
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer()
train_x = transformer.fit_transform(train_x)
test_x = transformer.transform(test_x)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
W = tf.Variable( tf.random.normal( [4,1] ) )
b = tf.Variable( tf.random.normal( [1] ) ) 
    



2.Define a function to calculate prediction

In [0]:
@tf.function
def prediction(train_x, W, b):
    y_pred = tf.add(tf.matmul(train_x, W), b)
    return y_pred

3.Loss (Cost) Function [Mean square error]

In [0]:
@tf.function
def loss(train_y, y_predicted):
    diff = train_y - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:

@tf.function
def train(train_x, train_y, w, b, learning_rate=0.01):
    
    # Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        t.watch([w,b])
        current_prediction = prediction(train_x, w, b)
        current_loss = loss(train_y, current_prediction)
    
    # Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    # Update Weights and Bias
    w = w - learning_rate * dw
    b = b - learning_rate * db
    
    return w, b
    

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [100]:
for i in range(100):    
    W, b = train(train_x, train_y, W, b)
    print('Current Training Loss on iteration', i, ' -- ', loss(train_y, prediction(train_x, W, b)).numpy())

('Current Training Loss on iteration', 0, ' -- ', 227.62114)
('Current Training Loss on iteration', 1, ' -- ', 226.91733)
('Current Training Loss on iteration', 2, ' -- ', 226.26868)
('Current Training Loss on iteration', 3, ' -- ', 225.67084)
('Current Training Loss on iteration', 4, ' -- ', 225.1199)
('Current Training Loss on iteration', 5, ' -- ', 224.61218)
('Current Training Loss on iteration', 6, ' -- ', 224.14426)
('Current Training Loss on iteration', 7, ' -- ', 223.71298)
('Current Training Loss on iteration', 8, ' -- ', 223.3156)
('Current Training Loss on iteration', 9, ' -- ', 222.94931)
('Current Training Loss on iteration', 10, ' -- ', 222.61172)
('Current Training Loss on iteration', 11, ' -- ', 222.30064)
('Current Training Loss on iteration', 12, ' -- ', 222.01392)
('Current Training Loss on iteration', 13, ' -- ', 221.74965)
('Current Training Loss on iteration', 14, ' -- ', 221.5062)
('Current Training Loss on iteration', 15, ' -- ', 221.28172)
('Current Training Lo

In [101]:
for i in range(100):    
    W, b = train(train_x, train_y, W, b)
    if i%5 == 0:
        print('Current Training Loss on iteration', i, ' -- ', loss(train_y, prediction(train_x, W, b)).numpy())
    

('Current Training Loss on iteration', 0, ' -- ', 218.64607)
('Current Training Loss on iteration', 5, ' -- ', 218.64525)
('Current Training Loss on iteration', 10, ' -- ', 218.64467)
('Current Training Loss on iteration', 15, ' -- ', 218.64426)
('Current Training Loss on iteration', 20, ' -- ', 218.64401)
('Current Training Loss on iteration', 25, ' -- ', 218.64386)
('Current Training Loss on iteration', 30, ' -- ', 218.64372)
('Current Training Loss on iteration', 35, ' -- ', 218.64366)
('Current Training Loss on iteration', 40, ' -- ', 218.64362)
('Current Training Loss on iteration', 45, ' -- ', 218.64357)
('Current Training Loss on iteration', 50, ' -- ', 218.64355)
('Current Training Loss on iteration', 55, ' -- ', 218.64355)
('Current Training Loss on iteration', 60, ' -- ', 218.64352)
('Current Training Loss on iteration', 65, ' -- ', 218.64352)
('Current Training Loss on iteration', 70, ' -- ', 218.64352)
('Current Training Loss on iteration', 75, ' -- ', 218.64352)
('Current 

### Get the shapes and values of W and b

In [102]:
print ('shape of W : ', W)

('shape of W : ', <tf.Tensor: id=3904, shape=(4, 1), dtype=float32, numpy=
array([[2.897813  ],
       [0.89226735],
       [1.7017184 ],
       [1.2495164 ]], dtype=float32)>)


In [103]:
print ('shape of W : ', b)

('shape of W : ', <tf.Tensor: id=3905, shape=(1,), dtype=float32, numpy=array([2.2829044], dtype=float32)>)


### Model Prediction on 1st Examples in Test Dataset

In [104]:
pred=prediction(test_x,W,b)
pred

<tf.Tensor: id=3915, shape=(300, 1), dtype=float32, numpy=
array([[5.647792 ],
       [5.653122 ],
       [5.656762 ],
       [5.633687 ],
       [5.651629 ],
       [5.64215  ],
       [5.656746 ],
       [5.6420965],
       [5.656914 ],
       [5.656017 ],
       [5.653165 ],
       [5.6450624],
       [5.635749 ],
       [5.6477776],
       [5.652419 ],
       [5.6417866],
       [5.6587667],
       [5.6528955],
       [5.645602 ],
       [5.6380444],
       [5.6579742],
       [5.6488724],
       [5.658024 ],
       [5.6391115],
       [5.653619 ],
       [5.64268  ],
       [5.6487217],
       [5.656662 ],
       [5.6516123],
       [5.644779 ],
       [5.6640053],
       [5.656612 ],
       [5.652178 ],
       [5.6526585],
       [5.6423817],
       [5.650836 ],
       [5.6480055],
       [5.6495094],
       [5.656887 ],
       [5.639617 ],
       [5.65985  ],
       [5.640571 ],
       [5.6530933],
       [5.649042 ],
       [5.65498  ],
       [5.635691 ],
       [5.6538935],
 

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
Irisd = pd.read_csv("/content/drive/My Drive/11_Iris.csv", delimiter=",")

In [111]:
Irisd.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [112]:
Irisd.shape

(150, 6)

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [0]:
# import labelencoder
from sklearn.preprocessing import LabelEncoder
# instantiate labelencoder object
le = LabelEncoder()
# apply le on categorical feature columns
Irisd.Species = le.fit_transform(Irisd.Species)

In [0]:
y_iris = tf.keras.utils.to_categorical(Irisd.Species, num_classes=3)

### Splitting the data into feature set and target set

In [119]:
X_iris = Irisd.iloc[:,1:5]
#y_iris = iris.Species
y_iris.shape

(150, 3)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
# Initialize Sequential model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(4, input_dim=4, kernel_initializer='normal', activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:

# Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

# Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X_iris, y_iris, test_size=0.20,random_state=3)

### Model Training 

In [123]:
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=30)

Train on 120 samples, validate on 30 samples
Epoch 1/30
120/120 [==============================] - 0s 3ms/sample - loss: 1.1048 - accuracy: 0.3833 - val_loss: 1.0853 - val_accuracy: 0.3333
Epoch 2/30
120/120 [==============================] - 0s 82us/sample - loss: 1.0835 - accuracy: 0.3500 - val_loss: 1.0834 - val_accuracy: 0.6667
Epoch 3/30
120/120 [==============================] - 0s 89us/sample - loss: 1.0727 - accuracy: 0.5333 - val_loss: 1.0631 - val_accuracy: 0.3333
Epoch 4/30
120/120 [==============================] - 0s 85us/sample - loss: 1.0603 - accuracy: 0.4833 - val_loss: 1.0503 - val_accuracy: 0.6000
Epoch 5/30
120/120 [==============================] - 0s 91us/sample - loss: 1.0468 - accuracy: 0.4250 - val_loss: 1.0398 - val_accuracy: 0.6333
Epoch 6/30
120/120 [==============================] - 0s 87us/sample - loss: 1.0365 - accuracy: 0.5833 - val_loss: 1.0348 - val_accuracy: 0.3667
Epoch 7/30
120/120 [==============================] - 0s 82us/sample - loss: 1.0264 - 

### Model Prediction

In [124]:
model.predict(testX)

array([[0.7869151 , 0.14342721, 0.06965768],
       [0.76611835, 0.15528665, 0.07859503],
       [0.8174351 , 0.1255271 , 0.05703775],
       [0.76474947, 0.15605798, 0.07919264],
       [0.89645654, 0.07608492, 0.02745853],
       [0.24852958, 0.36011386, 0.39135656],
       [0.2709083 , 0.35625154, 0.37284023],
       [0.7989696 , 0.13642927, 0.06460112],
       [0.24852958, 0.36011386, 0.39135656],
       [0.36153147, 0.33387664, 0.30459192],
       [0.32833046, 0.3431725 , 0.32849705],
       [0.76757634, 0.1544638 , 0.07795977],
       [0.31155038, 0.34741715, 0.34103248],
       [0.24852958, 0.36011386, 0.39135656],
       [0.24852958, 0.36011386, 0.39135656],
       [0.83349115, 0.11586107, 0.05064777],
       [0.24852958, 0.36011386, 0.39135656],
       [0.24852958, 0.36011386, 0.39135656],
       [0.24852958, 0.36011386, 0.39135656],
       [0.7877929 , 0.14292076, 0.06928635],
       [0.24852958, 0.36011386, 0.39135656],
       [0.24852958, 0.36011386, 0.39135656],
       [0.

### Save the Model

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?